#### This notebook is designed to demonstrate the functioning of the `exam` command line tool by simulating treatment effect, WTP, and predicted treatment data, then running through the probability assignment and effect estimation portions of the package.

In [1]:
import numpy as np
import pandas as pd

## Simulate Data

We will simulate the effect of 2 treatments (and 1 control group), along with some randomly generated control variables, for a sample of 1000.

In [2]:
np.random.seed(2)
n_treatments = 2

effects = np.random.choice(range(1,50), n_treatments)
effects = np.append(0, effects)
controls = np.random.uniform(-5, 5, size = (1000, 3))
control_effects = np.random.choice(range(-5,5), replace = False, size = 3)
error = np.random.uniform(size = 1000)
Y = np.sum(controls*control_effects, axis=1)[:,np.newaxis] + np.repeat(effects[np.newaxis,:], 1000, axis=0) + error[:,np.newaxis]

In [3]:
Y[:5]

array([[-2.78148316, 38.21851684, 13.21851684],
       [17.09378918, 58.09378918, 33.09378918],
       [17.34526741, 58.34526741, 33.34526741],
       [15.10134814, 56.10134814, 31.10134814],
       [-1.04175403, 39.95824597, 14.95824597]])

WTP (willingness to pay) and PTE (predicted treatment effects) will be computed as a function of the true effects. Specifically, WTP will be the quintile index of the control effects on outcome and PTE will be half of the true treatment effects.

In [5]:
control_outcomes = controls*control_effects

In [6]:
from scipy import stats
wtp = np.array([stats.percentileofscore(control_outcomes.flatten(), x) for x in control_outcomes.flatten()]).reshape(1000,3)
wtp = (np.ceil(wtp/20)*20).astype(int)

In [7]:
pte = np.repeat(effects[np.newaxis,:]/2, 1000, axis=0)

In [8]:
print(wtp[:5])
print(pte[:5])

[[ 40  60  80]
 [ 60  80 100]
 [ 60  80 100]
 [ 60  60 100]
 [ 60 100  20]]
[[ 0.  20.5  8. ]
 [ 0.  20.5  8. ]
 [ 0.  20.5  8. ]
 [ 0.  20.5  8. ]
 [ 0.  20.5  8. ]]


## Compute treatment probabilities

The `exam` compute_probs.py command line tool requires a saved CSV file containing the WTP columns, followed by the PTE columns. The indices of the file will be assumed to be the subject ids, and (unless otherwise specified) the column names will be assumed to be the treatment labels. 

In [9]:
# Save WTP and PTE data
tot_arr = np.column_stack([wtp, pte])
df = pd.DataFrame(tot_arr, columns = ["control", "treatment1", "treatment2"]*2, index = [f"subject{i}" for i in range(1000)])
df.to_csv("data/sim_data.csv")

With just WTP and PTE, we can now compute our welfare-optimal treatment probabilities. `compute_probs.py` can take a variety of parameters, but the only required one is our saved data. 

In [169]:
!python ../exam/compute_probs.py -h

usage: compute_probs.py [-h] --data DATA [--output OUTPUT]
                        [--assign_output ASSIGN_OUTPUT]
                        [--capacity [CAPACITY ...]] [--pbound PBOUND]
                        [--error ERROR] [--iterations ITERATIONS]
                        [--budget BUDGET] [--subject_budgets SUBJECT_BUDGETS]
                        [--round ROUND] [--labels [LABELS ...]] [--index]

optional arguments:
  -h, --help            show this help message and exit
  --data DATA           Path to CSV containing WTP and PTE, in that order. If
                        --index flag toggled, the first column will be assumed
                        to be the index containing subject ids. The columns
                        will be taken to be treatment labels, unless otherwise
                        specified in --labels.
  --output OUTPUT       Path to save CSV file with treatment probabilities.
  --assign_output ASSIGN_OUTPUT
                        Path to save CSV file with tr

We will save the computed probabilities to "data/sim_data_probs.csv" and our assigned treatments to "data/sim_data_assignments.csv". Note that our saved data has an index column with subject ids, so we will need to toggle the `--index` flag. We will also set a probability bound so that no one subject is guaranteed a particular treatment. 

In [177]:
!python ../exam/compute_probs.py --data data/sim_data.csv --output data/sim_data_probs.csv --assign_output data/sim_data_assignments.csv --index --pbound 0.2

compute_probs: Parameters
--------------------------------------------------
# treatments: 3
# subjects: 1000
capacity: [334 333 333]
epsilon-bound: 0.2
error clearing threshold: 0.01
iterations threshold: 20
budget type: constant

get_clearing_error: Clearing error: 0.04355442084763332
get_clearing_error: Clearing error: 0.04928635935140773
get_clearing_error: Clearing error: 0.04978825685318274
get_clearing_error: Clearing error: 0.049434905522447246
get_clearing_error: Clearing error: 0.04753579242932104
get_clearing_error: Clearing error: 0.0498272609021343
get_clearing_error: Clearing error: 0.04857309080125088
get_clearing_error: Clearing error: 0.05085809874360182
get_clearing_error: Clearing error: 0.05020297753601493
get_clearing_error: Clearing error: 0.04899823341150435
get_clearing_error: Clearing error: 0.049399708296320774
get_clearing_error: Clearing error: 0.050766194443281426
get_clearing_error: Clearing error: 0.057629701241564704
get_clearing_error: Clearing error: 0

get_clearing_error: Clearing error: 0.011803056877826276
get_clearing_error: Clearing error: 0.005532730059489983
Minimum clearing error: 0.005532730059489983
Alpha_star: -11.875095812703094
Beta star: [-207.03287382 -116.36584446  355.7919686 ]
Rounding probabilities to 2 decimals...
[[0.2006006  0.30348094 0.49591846]
 [0.2006006  0.30348094 0.49591846]
 [0.2006006  0.30348094 0.49591846]
 ...
 [0.2006006  0.5993994  0.2       ]
 [0.2006006  0.5993994  0.2       ]
 [0.34351184 0.2        0.45648816]]
[[0.2  0.3  0.5 ]
 [0.2  0.3  0.5 ]
 [0.2  0.3  0.5 ]
 ...
 [0.2  0.6  0.2 ]
 [0.2  0.6  0.2 ]
 [0.34 0.2  0.46]]
Treatment probabilities saved to: data/sim_data_probs.csv
Treatment assignments saved to: data/sim_data_assignments.csv


In [62]:
probs = pd.read_csv("data/sim_data_probs.csv", index_col = 0)
print(probs.head())
assignments = pd.read_csv("data/sim_data_assignments.csv", index_col = 0)
print(assignments.head())

           control  treatment1  treatment2
subject0  0.200601    0.307711    0.491688
subject1  0.200601    0.307711    0.491688
subject2  0.200601    0.307711    0.491688
subject3  0.200601    0.307711    0.491688
subject4  0.200601    0.599399    0.200000
          assignment
subject0  treatment2
subject1  treatment2
subject2     control
subject3  treatment2
subject4  treatment1


## Estimate Treatment Effects

The treatment effect estimation tool requires two inputs: 
1. a path to the post-experiment saved data with columns ordered by [outcome (Y), treatment assignment (D), X] where X is any number of control variables to include in the effect estimation. 
2. a path to the saved treatment probabilities 

In [63]:
# Get outcomes based on assignments
assignments["assign_ind"] = assignments.assignment.astype('category').cat.reorder_categories(['control', 'treatment1', 'treatment2'], ordered=True)
assignments["assign_ind"] = assignments.assign_ind.cat.codes
outcomes = Y[(np.arange(1000), assignments.assign_ind.to_numpy().flatten())]
print(Y[:5])
print(outcomes[:5])

[[-2.78148316 38.21851684 13.21851684]
 [17.09378918 58.09378918 33.09378918]
 [17.34526741 58.34526741 33.34526741]
 [15.10134814 56.10134814 31.10134814]
 [-1.04175403 39.95824597 14.95824597]]
[13.21851684 33.09378918 17.34526741 31.10134814 39.95824597]


In [64]:
# Bind simulated outcomes Y with treatment assignments and controls
y_df = pd.DataFrame(outcomes)
control_df = pd.DataFrame(controls, columns = ['X1', 'X2', 'X3'])
assignments = assignments.reset_index()
outcome_df = pd.concat([y_df, assignments.assignment, control_df], axis = 1)
outcome_df.head()

,0,assignment,X1,X2,X3
0,13.218517,treatment2,-4.740738,0.496625,-0.646776
1,33.093789,treatment2,-0.796322,-1.696652,-2.953514
2,17.345267,control,1.192710,-2.003453,-2.331727
3,31.101348,treatment2,1.211338,0.291421,-3.654201
4,39.958246,treatment1,0.135781,-3.155601,2.853351


In [65]:
outcome_df.to_csv("data/sim_data_outcomes.csv")

Now we can run treatment effect estimation using our saved outcome data and (previously) saved probability data. Note that both files have an index column with subject ids, so we will need to toggle the `--dindex` and `--pindex` flags. 

In [66]:
!python ../exam/estimate_effects.py -h

usage: estimate_effects.py [-h] --data DATA --probs PROBS [--output OUTPUT]
                           [--control CONTROL] [--method METHOD] [--dindex]
                           [--pindex] [--noverb]

optional arguments:
  -h, --help         show this help message and exit
  --data DATA        Path to CSV containing experimental outcomes Y, treatment
                     assignments D, and any controls X, in that order.
  --probs PROBS      Path to CSV containing treatment probabilities.
  --output OUTPUT    Path to save CSV file with estimation output.
  --control CONTROL  Name of column of control treatment.
  --method METHOD    Method to apply to estimation. Either 'matched' or
                     'single'.
  --dindex           Flag to indicate whether 'data' file has an index column.
  --pindex           Flag to indicate whether 'probs' file has index column.
  --noverb           Flag to turn off printing estimation results to stdout.


As with the API, the command line tool offers both estimation methods. The default will be to estimate the unbiased ATE with the propensity score matched estimator. 

**Note:** The package will automatically check whether certain subpopulation regressions are rank-deficient and drop them from estimation. This may skew the estimation results if your computed propensity vectors are not coarse enough.

In [168]:
!python ../exam/estimate_effects.py --data data/sim_data_outcomes.csv --probs data/sim_data_probs.csv --output data/sim_data_effects.csv --dindex --pindex

---------------------------------------------------------------------------
ATE estimation method: propensity subpopulation regressions
---------------------------------------------------------------------------
/Users/rl874/opt/anaconda3/envs/exam/lib/python3.9/site-packages/pandas/core/indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
Total rank-deficient samples dropped: 174
Estimated treatment effects:
treatment1    41.029376
treatment2    15.995976
dtype: float64
P-values:
treatment1    0.0
treatment2    0.0
Name: p-value, dtype: float64


We can compare the estimates against our true effects

In [151]:
effects

array([ 0, 41, 16])